# 0. Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   
import warnings

sns.set_theme(style="darkgrid")
warnings.filterwarnings('ignore')

# 2. Data Preprocessing

In [ ]:
df = pd.read_csv('data/FT1000_v0.csv')
df.head()

,Rank,Name,in 2021 ranking,in 2020 ranking,Country,Sector,Absolute Growth Rate %,Compound Annual Growth Rate (CAGR) %,Revenue 2020 (€),Revenue 2017 (€),...,Unnamed: 990,Unnamed: 991,Unnamed: 992,Unnamed: 993,Unnamed: 994,Unnamed: 995,Unnamed: 996,Unnamed: 997,Unnamed: 998,Unnamed: 999
0,1.0,Swappie,No,No,Finland,Technology,19152.82,477.43,97611814.0,507000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Kilo Health,No,No,Lithuania,Health,16541.73,450.05,57318766.0,344428.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,OCI,No,No,UK,Financial Services,13133.13,409.59,568322073.0,4325512.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,OnlyFans,No,No,UK,Technology,11928.05,393.63,316732986.0,2652185.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Enpal,No,No,Germany,Energy,11441.27,386.88,56109613.0,486165.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.5. Outliers
Ở phần này nhóm tiến hành kiểm tra outliers ở các cốt. Cột `Rank` và `Name` là key value, do đó không kiểm tra outliers trên các cột này

### 2.5.1. Các cột categorical

In [ ]:
df['in 2020 ranking'].value_counts()

In [ ]:
df['in 2021 ranking'].value_counts()

No     694
Yes    306
Name: in 2021 ranking, dtype: int64

In [ ]:
df['Country'].value_counts()

Italy              235
Germany            194
UK                 155
France             147
Spain               49
Poland              34
Sweden              28
The Netherlands     19
Hungary             13
Norway              11
Finland             11
Lithuania           11
Czech Republic      11
Belgium             10
Romania              8
Croatia              7
Slovakia             7
Bulgaria             7
Portugal             7
Denmark              6
Greece               5
Austria              5
Estonia              4
Ireland              4
Switzerland          4
Cyprus               3
Slovenia             2
Luxembourg           1
Latvia               1
Liechtenstein        1
Name: Country, dtype: int64

Có tới 30 quốc gia có công ty tham gia xếp hạng nhưng chủ yếu chỉ tập trung ở top 5, 6 quốc gia đầu tiên, phần lớn còn lại nằm ở mức dưới 10 công ty một quốc gia. Nhiều quốc gia chỉ có từ 1 tới 2 công ty tham gia. Sau khi bàn bạc, nhóm quyết định gom nhóm các quốc gia có từ 5 công ty trở xuống vào một nhóm, gọi là `Other` và lưu vào cột `Country1`

In [ ]:
other_country = df['Country'].value_counts()[df['Country'].value_counts() <= 5].index
df['Country1'] = df['Country']
df['Country1'].replace(other_country, 'Other', inplace = True)